In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp downloaderlabeler
# all_local

In [ ]:
!pip install .[dev]

Defaulting to user installation because normal site-packages is not writeable
Processing /home/cedar/source/steroids-or-not
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


  Created wheel for steroidsornot: filename=steroidsornot-0.0.1-py3-none-any.whl size=7143 sha256=964bf99e73135f29e72766068c38bc753d16b348885b513719a617a17d6f47e9
  Stored in directory: /home/cedar/.cache/pip/wheels/8b/e8/f0/812780b69e692fdae0be65743cbacdc466f0d0c0cc3446797e
Successfully built steroidsornot
  Attempting uninstall: steroidsornot
    Found existing installation: steroidsornot 0.0.1
    Uninstalling steroidsornot-0.0.1:
      Successfully uninstalled steroidsornot-0.0.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
import pickle
from steroidsornot.submissionshandler import SubmissionsHandler

In [ ]:
from fastcore.test import *

In [ ]:
class DownloaderLabeler():
    def __init__(self, path):
        self.submissions_handler = SubmissionsHandler(path)
        
        self._setup_buttons()
        
        self._main_loop()
        
    def _main_loop(self):
        pass
        
    def _next_submission():
        pass

        
    def _setup_buttons(self):
        self.natural_button = Button(description='natural', layout=Layout(border = '2px solid green'))
        self.uncertain_button = Button(description='uncertain', layout=Layout(border = '2px solid brown'))
        self.steroids_button = Button(description='steroids', layout=Layout(border = '2px solid red'))
        self.irrelevant_button = Button(description='irrelevant', layout=Layout(border = '2px solid khaki'))
        self.box = widgets.HBox([self.natural_button,
                                 self.uncertain_button,
                                 self.steroids_button,
                                 self.irrelevant_button,
                                ])
        display(self.box)       

In [ ]:
downloader_labeler = DownloaderLabeler('data/downloader.pkl')

Unpickled 41725 objects.
16260 are mostly useful.

This should show no [deleted] or [removed] entries:
[('', 16260)]


In [ ]:
downloader_labeler.submissions_handler.submissions[0].json()

AttributeError: 'dict' object has no attribute 'json'